# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Arxiv has 66 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.20775


extracting tarball to tmp_2511.20775...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2511.20775/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'appendix' from 'tmp_2511.20775/appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'acknowledgment' from 'tmp_2511.20775/acknowledgment.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxi

...

 done.


Found 70 bibliographic references in tmp_2511.20856/AngularSystematics.bbl.
Issues with the citations
syntax error in line 25: '=' expected
Retrieving document from  https://arxiv.org/e-print/2511.20970


extracting tarball to tmp_2511.20970...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.21243
extracting tarball to tmp_2511.21243...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20775-b31b1b.svg)](https://arxiv.org/abs/2511.20775) | **Ashes of Creation: JWST Uncovers Silicate Dust in Massive Star Clusters**  |
|| D. Maschmann, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-11-27*|
|*Comments*| *38 Pages, 13Figures, submitted to APJ*|
|**Abstract**|            Dust production is a fundamental aspect of the baryonic cycle of star formation. It is known that dust is injected into the interstellar medium during early star formation by supernovae and later on by evolved stars. From individual objects, these mechanisms are well understood, but the overall dust production in star clusters at different evolutionary stages is still challenging to quantify. We present 22 massive (> 105M$_{\odot}$) extra galactic star clusters with ages between 3 and 100 Myr exhibiting a compact dust morphology seen with JWST-MIRI. We only find PAH features associated with one star cluster and nineteen have already cleared themselves from their natal dust. Their main characteristic is a significant enhancement at 10${\mu}$m, which is likely due to silicate emission and cannot be explained by ionized gas. We discuss several possible explanations including dust production from evolved stars such as red super giants, more exotic star types like yellow hypergiants and luminous blue variable stars. Stochastic dust injection from supernovae or a single supernova in dense gas can also create significant silicate emission. However, for this scenario secondary tracers such as a X-ray signal are expected which we only observe in three star clusters. We find the most luminous 10${\mu}$m emitter to be the three most massive star clusters (> 106M$_{\odot}$) which is at least a magnitude stronger than any known stellar sources indicating a rare mechanism that only appears at extreme masses and a short lifetime.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20856-b31b1b.svg)](https://arxiv.org/abs/2511.20856) | **Euclid preparation. Controlling angular systematics in the Euclid spectroscopic galaxy sample**  |
|| E. Collaboration, et al. -- incl., <mark>Y. Wang</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-11-27*|
|*Comments*| *submitted to A&A; 34 pages, 18 figures, 4 appendices*|
|**Abstract**|            We present the strategy to identify and mitigate potential sources of angular systematics in the Euclid spectroscopic galaxy survey, and we quantify their impact on galaxy clustering measurements and cosmological parameter estimation. We first survey the Euclid processing pipeline to identify all evident, potential sources of systematics, and classify them into two broad classes: angular systematics, which modulate the galaxy number density across the sky, and catastrophic redshift errors, which lead to interlopers in the galaxy sample. We then use simulated spectroscopic surveys to test our ability to mitigate angular systematics by constructing a random catalogue that represents the visibility mask of the survey; this is a dense set of intrinsically unclustered objects, subject to the same selection effects as the data catalogue. The construction of this random catalogue relies on a detection model, which gives the probability of reliably measuring the galaxy redshift as a function of the signal-to-noise ratio (S/N) of its emission lines. We demonstrate that, in the ideal case of a perfect knowledge of the visibility mask, the galaxy power spectrum in the presence of systematics is recovered, to within sub-percent accuracy, by convolving a theory power spectrum with a window function obtained from the random catalogue itself. In the case of only approximate knowledge of the visibility mask, we test the stability of power spectrum measurements and cosmological parameter posteriors by using perturbed versions of the random catalogue. We find that significant effects are limited to very large scales, and parameter estimation remains robust, with the most impacting effects being connected to the calibration of the detection model.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20970-b31b1b.svg)](https://arxiv.org/abs/2511.20970) | **The diverse morphology of gravitational wave signals from merging neutron-star white-dwarf binaries**  |
|| S. Yu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-11-27*|
|*Comments*| *13 pages, 7 figures, accepted for publication in MNRAS*|
|**Abstract**|            In sufficiently compact neutron star-white dwarf (NSWD) binary systems, orbital decay means the white dwarf eventually fills its shrinking Roche lobe, initiating a phase of mass transfer. The exchange of angular momentum-both internal and external-plays a critical role in determining the binary's evolutionary outcome. For neutron stars with relatively low magnetic fields and spin frequencies, whether the orbital separation continues to shrink depends on the interplay between gravitational wave (GW) radiation and mass transfer dynamics. We compute the orbital evolution of NSWD binaries across a broad parameter space, incorporating four key variables. Our results reveal distinct boundaries in the NS-WD mass-mass diagram: binaries with white dwarf masses above these thresholds undergo rapid orbital decay and direct coalescence. The dependence of these boundaries on system parameters indicates that Roche-lobe-filling NSWD binaries can follow multiple evolutionary pathways -- a phenomenon we refer to as branched or polymorphic evolution. NSWD binary systems emit strong and diverse GW signals, many of which would be detectable by space-based GW observatories. The morphology of the evolving GW waveform provides a direct diagnostic for the NSWD binary configuration, including any contribution from an accretion disk. Our models can provide critical waveform templates for identifying merging binary signals in real-time GW data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.21243-b31b1b.svg)](https://arxiv.org/abs/2511.21243) | **The Faintest, Extremely Variable X-ray Tidal Disruption Event from a Supermassive Black Hole Binary?**  |
|| M. Huang, et al. -- incl., <mark>Y. Wang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-11-27*|
|*Comments*| *31 pages, 6 figures, 1 table, accepted by The Innovation*|
|**Abstract**|            Tidal disruption events (TDEs), which occur when stars enter the tidal radii of supermassive black holes (SMBHs) and are subsequently torn apart by their tidal forces, represent intriguing phenomena that stimulate growing research interest and pose an increasing number of puzzles in the era of time-domain astronomy. Here we report an unusual X-ray transient, XID 935, discovered in the 7 Ms Chandra Deep Field-South, the deepest X-ray survey ever. XID 935 experienced an overall X-ray dimming by a factor of more than 40 between 1999 and 2016. Not monotonically decreasing during this period, its X-ray luminosity increased by a factor $> 27$ within 2 months, from $L_{\rm 0.5-7\ keV}<10^{40.87}$ erg s$^{-1}$ (10 October 2014 -- 4 January 2015) to $L_{\rm 0.5-7\ keV}=10^{42.31\pm 0.20}$ erg s$^{-1}$ (16 March 2015). The X-ray position of XID 935 is located at the center of its host galaxy with a spectroscopic redshift of 0.251, whose optical spectra do not display emission characteristics associated with an active galactic nucleus. The peak 0.5--2.0 keV flux is the faintest among all the X-ray-selected TDE candidates to date. Thanks to a total exposure of $\sim 9.5$ Ms in the X-ray bands, we manage to secure relatively well-sampled, 20-year-long X-ray light curves of this deepest X-ray-selected TDE candidate. We find that a partial TDE model could not explain the main declining trend. An SMBH binary TDE model is in acceptable accordance with the light curves of XID 935; however, it fails to match short-timescale fluctuations exactly. Therefore, the exceptional observational features of XID 935 provide a key benchmark for refining quantitative TDE models and simulations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.20775/./excess_mag_final.png', 'tmp_2511.20775/./example_apert_profile_corr_F1000W.png', 'tmp_2511.20775/./sed_lum_plot.png']
copying  tmp_2511.20775/./excess_mag_final.png to _build/html/
copying  tmp_2511.20775/./example_apert_profile_corr_F1000W.png to _build/html/
copying  tmp_2511.20775/./sed_lum_plot.png to _build/html/
exported in  _build/html/2511.20775.md
    + _build/html/tmp_2511.20775/./excess_mag_final.png
    + _build/html/tmp_2511.20775/./example_apert_profile_corr_F1000W.png
    + _build/html/tmp_2511.20775/./sed_lum_plot.png
found figures ['tmp_2511.20856/./Figures/shuf_pk_model.png', 'tmp_2511.20856/./Figures/detmodel.png', 'tmp_2511.20856/./Figures/detmodel_scatter.png', 'tmp_2511.20856/./Figures/shuf_pk3_realistic.png']
copying  tmp_2511.20856/./Figures/shuf_pk_model.png to _build/html/
copying  tmp_2511.20856/./Figures/detmodel.png to _build/html/
copying  tmp_2511.20856/./Figures/detmodel_scatter.png to _build/html/
copying  tmp_2511.20856

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Arizona}{\affil{Steward Observatory, University of Arizona, Tucson, AZ 85721, USA$
$}}$
$\newcommand{\GEMINI}{\affil{Gemini Observatory/NSF NOIRLab, 950 N. Cherry Avenue, Tucson, AZ, 85719, USA$
$}}$
$\newcommand{\ASCL}{\affil{Astrophysics Source Code Librar$
$y, Michigan Technological University, 1400 Townsend Drive, Houghton, MI 49931}}$
$\newcommand{\OSU}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{\Alberta}{\affil{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\ANU}{\affil{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\IPARCOS}{\affil{Instituto de Física de Partículas y del Cosmos, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\IPAC}{\affil{Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{\Caltech}{\affil{Caltech, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{\Carnegie}{\affil{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{ÇAPP}{\affil{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\CfA}{\affil{Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{\CITEVA}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\CNRS}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\ESO}{\affil{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}}$
$\newcommand{\Heidelberg}{\affil{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\Cologne}{\affil{I. Physikalisches Institut, Universität zu Köln, Zülpicher Str. 77, D-50937 Köln, Germany}}$
$\newcommand{\ITA}{\affil{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affil{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\ICRAR}{\affil{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{\IRAM}{\affil{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}}$
$\newcommand{\IRAP}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UPS}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\JHU}{\affil{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{\Leiden}{\affil{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}}$
$\newcommand{\Maryland}{\affil{Department of Astronomy, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{\MPE}{\affil{Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{\MPIA}{\affil{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{\Nagoya}{\affil{Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}}$
$\newcommand{\NRAO}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{\OAN}{\affil{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{\ObsParis}{\affil{Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}}$
$\newcommand{\Princeton}{\affil{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}}$
$\newcommand{\UToledo}{\affil{University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}}$
$\newcommand{\RitterToledo}{\affil{Ritter Astrophysical Research Center, University of Toledo, 2801 W. Bancroft St., MS 113, Toledo, OH, 43606}}$
$\newcommand{\Toulouse}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UBonn}{\affil{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}}$
$\newcommand{\UChile}{\affil{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{\UCM}{\affil{Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\UCSD}{\affil{Department of Astronomy and Astrophysics,  University of California,\ San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\ULyon}{\affil{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}}$
$\newcommand{\UMass}{\affil{University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}}$
$\newcommand{\UWyoming}{\affil{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\LAM}{\affil{$
$Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille),  F-13388 Marseille,$
$France}}$
$\newcommand{\UHawaii}{\affil{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}}$
$\newcommand{\UGent}{\affil{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\IPARC}{\affil{Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{\STScI}{\affil{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\STScIESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\ESA}{\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\McMaster}{\affil{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\INAF}{\affil{INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}}$
$\newcommand{\Sydney}{\affil{Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}}$
$\newcommand{\UA}{\affil{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\LERMA}{\affil{Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universités, 75014 Paris}}$
$\newcommand{\SAIMSU}{\affil{Sternberg Astronomical Institute, Lomonosov Moscow State University, Universitetsky pr. 13, 119234 Moscow, Russia}}$
$\newcommand{\UTA}{\affil{Instituto de Alta Investigación, Universidad de Tarapacá, Casilla 7D, Arica, Chile}}$
$\newcommand{\IAC}{\affil{Instituto de Astrofísica de Canarias, C/ Vía Láctea s/n, E-38205, La Laguna, Spain}}$
$\newcommand{\UNAM}{\affil{Instituto de Astronomía, Universidad Nacional Autónoma de México, Unidad Académica en Ensenada, Km 103 Carr. Tijuana−Ensenada, Ensenada, B.C.,$
$C.P. 22860, México}}$
$\newcommand{\ULL}{\affil{Departamento de Astrofísica, Universidad de La Laguna, Av. del Astrofísico Francisco Sánchez s/n, E-38206, La Laguna, Spain}}$
$\newcommand{ÅPF}{\altaffiliation{NSF Astronomy and Astrophysics Postdoctoral Fellow}}$
$\newcommand{\DECRA}{\altaffiliation{ARC DECRA Fellow}}$
$\newcommand{\Oxford}{\affil{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}}$
$\newcommand{\wesleyan}{\affil{Astronomy Department and Van Vleck Observatory, Wesleyan University, 96 Foss Hill Drive, Middletown, CT 06459, USA}}$
$\newcommand{\PLATA}{\affil{Instituto de Astrofísica de La Plata, CONICET--UNLP, Paseo del Bosque S/N, B1900FWA La Plata, Argentina }}$
$\newcommand{\ARC}{\affil{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\UVirginia}{\affil{University of Virginia Astronomy Department, 530 McCormick Road, Charlottesville, VA 22904, USA}}$
$\newcommand{\UniCA}{\affil{Université Côte d'Azur, Observatoire de la Côte d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}}$
$\newcommand{\CamIoA}{\affil{Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}}$
$\newcommand{\KICC}{\affil{Kavli Institute for Cosmology Cambridge, Madingley Road, Cambridge CB3 0HA, UK}}$
$\newcommand{\UOA}{\affil{Department of Physics, University of Arkansas, Fayetteville, AR 72701, USA}}$
$\newcommand{\ACSPS}{\affil{Arkansas Center for Space and Planetary Sciences, University of Arkansas, Fayetteville, AR 72701, USA}}$
$\newcommand{\JBCA}{\affil{UK ALMA Regional Centre Node, Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, The University of Manchester, Oxford Road, Manchester M13 9PL, UK}}$
$\newcommand{\manchaster}{\affil{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, The University of Manchester, Oxford Road, Manchester M13 9PL, UK}}$
$\newcommand{\TAMU}{\affil{George P. and Cynthia W. Michell Institute for Fundamental Physics \& Astronomy, Texas A\&M University, College Station, TX, 77843, USA}}$
$\newcommand{\PMO}{\affil{Purple Mountain Observatory, Chinese Academy of Sciences, 10 Yuanhua Road, Nanjing 210023, China}}$
$\newcommand{\UNC}{\affil{Department of Physics and Astronomy, University of North Carolina, Chapel Hill, NC 27599-3255, US}}$</div>



<div id="title">

# Ashes of Creation: JWST Uncovers Silicate Dust in Massive Star Clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.20775-b31b1b.svg)](https://arxiv.org/abs/2511.20775)<mark>Appeared on: 2025-11-27</mark> -  _38 Pages, 13Figures, submitted to APJ_

</div>
<div id="authors">

D. Maschmann, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Dust production is a fundamental aspect of the baryonic cycle of star formation. It is known that dust is injected into the interstellar medium during early star formation by supernovae and later on by evolved stars. From individual objects, these mechanisms are well understood, but the overall dust production in star clusters at different evolutionary stages is still challenging to quantify. We present 22 massive ( $>10^5 M_{\odot}$ ) extra galactic star clusters with ages between 3 and 100 Myr exhibiting a compact dust morphology seen with JWST-MIRI. We only find PAH features associated with one star cluster and nineteen have already cleared themselves from their natal dust. Their main characteristic is a significant enhancement at $10\mu m$ , which is likely due to silicate emission and cannot be explained by ionized gas. We discuss several possible explanations including dust production from evolved stars such as red super giants, more exotic star types like yellow hypergiants and luminous blue variable stars. Stochastic dust injection from supernovae or a single supernova in dense gas can also create significant silicate emission. However, for this scenario secondary tracers such as a X-ray signal are expected which we only observe in three star clusters. We find the most luminous $10\mu m$ emitter to be the three most massive star clusters ( $>10^6 M_{\odot}$ ) which is at least a magnitude stronger than any known stellar sources indicating a rare mechanism that only appears at extreme masses and a short lifetime.

</div>

<div id="div_fig1">

<img src="tmp_2511.20775/./excess_mag_final.png" alt="Fig9" width="100%"/>

**Figure 9. -** $10\mu$m excess, E$_{\rm 10\mu m}$, as a function of the absolute AB magnitude in the F1000W band, M$_{\rm F1000W}$. We show all detected sources in gray that have S/N$\geq 3$ in the F770W, F1000W and F1130W MIRI bands. In crowded regions we show grayscale contours to better visualize the density.
 All cross-matched star clusters from maschmann_phangs-hst_2024 are marked with black diamonds and sources that do not show any optical counterpart are marked with black circles.
 The black dashed line indicates the maximal value of E$_{\rm 10\mu m}$ that can be predicted by simple dust models as described in detail in Section \ref{sec:model_predict}.
 The $10\mu$m-emitters selected for this work are highlighted by red diamonds.
 For reference, we indicate with purple contours PAH-emitters that have been selected in rodriguez_tracing_2025, and with red, green and blue stars the stars categorized by \citet[][in prep.]{hassani25}. Orange stars indicate SNRs from the catalogs of winkler_optical_2021 and li_discovery_2024, and yellow markers show known stars exhibiting strong silicate features.
 In order to visualize the SED shape of PAH emitters and $10\mu$m emitters we show each of these objects in the top left and right, respectively: We show a $5$\arcsec$\times5\$arcsec_rgb_-cutout of the MIRI bands F770W (blue), F1000W (green), F1130W (red) and their corresponding SED.
  (*fig:excess*)

</div>
<div id="div_fig2">

<img src="tmp_2511.20775/./example_apert_profile_corr_F1000W.png" alt="Fig4" width="100%"/>

**Figure 4. -** Schematic presentation of how we compute the aperture correction grid for the MIRI F1000W band. On the top left we show the simulated PSF which was super-sampled by a factor of 4. In the top middle panel we show the PSF convolved with a Moffat function of a ${\rm FWHM}=9.29 {\rm pix}$. With a black circle we indicate the aperture in which we measure the flux fraction. With colored rectangles we show the position of 12 slits in which we measure the source profiles presented in the top right panel and fit a Gaussian function to the part inside the aperture marked by black dashed lines. In the bottom panel we show the grid of aperture radii and the mean Gaussian standard deviations $\overline{\sigma}$ measured from the slit profiles. Each circle represents one measurement and in the background we present the interpolated values. With a black star we show the position of the estimated value presented with the top panels. (*fig:phot_explain*)

</div>
<div id="div_fig3">

<img src="tmp_2511.20775/./sed_lum_plot.png" alt="Fig1" width="100%"/>

**Figure 1. -** MIR SED in units of luminosity for all $10 \mu m$ emitting star clusters and selected stellar sources (See Sect. \ref{ssec:stars_milky}). We show the ISO spectra of the star systems $\eta$ Carinae, the YHG IRC+10420, the RSG HV 888 and the post-AGB star IRAS 18062+2410. We also show the SED points of the young star cluster Westerlund 1 observed with SOFIA.
 We display the SEDs of the star cluster sample with gray and highlight the three brightest star clusters with black thicker lines. (*fig:sed_lum*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.20775"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pigi}[1]{\textcolor{orange}{ [Pigi: #1]}}$
$\newcommand{\MYK}[1]{\textcolor{magenta}{ [#1]}}$
$\newcommand{\vx}{\ensuremath{\vec{x}}\xspace}$
$\newcommand{\vk}{\ensuremath{\vec{k}}\xspace}$
$\newcommand{\mc}[1]{\ensuremath{\mathcal{#1}}\xspace}$
$\newcommand{\nh}{\ensuremath{\hat{\vec n}}\xspace}$
$\newcommand{\dd}{\ensuremath{\mathrm{d}}\xspace}$
$\newcommand{\hmpc}{\ensuremath{h^{-1} \text{Mpc}}\xspace}$
$\newcommand{\mpc}{\ensuremath{\text{Mpc}}\xspace}$
$\newcommand{\kmpc}{\ensuremath{h \text{Mpc}^{-1}}\xspace}$
$\newcommand{\dens}{\ensuremath{h^{3} \mathrm{Mpc}^{-3}}\xspace}$
$\newcommand{\cgpc}{\ensuremath{h^{-3} \mathrm{Gpc}^3}\xspace}$
$\newcommand{\hgpc}{\ensuremath{h^{-1} \text{Gpc}}\xspace}$
$\newcommand{\Comp}{\ensuremath{{\cal{C}}}\xspace}$
$\newcommand{\Compa}{\ensuremath{\hat{\cal C}}\xspace}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}\xspace}$
$\newcommand{\pypelid}{\texttt{pypelid}\xspace}$
$\newcommand{\pinocchio}{\texttt{pinocchio}\xspace}$
$\newcommand{\healpix}{\texttt{healpix}\xspace}$
$\newcommand{\tdeg}{30◦ee\xspace}$
$\newcommand{\epixs}{e pix^{-1} s^{-1}\xspace}$
$\newcommand{\sqdeg}{deg^2\xspace}$
$\newcommand{\oiii}{[\ion{O}{III}]\xspace}$
$\newcommand{\oiiib}{[\ion{O}{III}]b\xspace}$
$\newcommand{\siii}{[\ion{S}{III}]\xspace}$
$\newcommand{\oii}{[\ion{O}{II}]\xspace}$
$\newcommand{\sii}{[\ion{S}{II}]\xspace}$
$\newcommand{\nii}{[\ion{N}{II}]\xspace}$
$\newcommand{\citepype}{Euclid Collaboration: Granett et al. (in prep.)\xspace}$
$\newcommand{\citeppype}{(Euclid Collaboration: Granett et al., in prep.)\xspace}$
$\newcommand{\citepk}{Euclid Collaboration: Salvalaggio et al. (in prep.)\xspace}$
$\newcommand{\citeppk}{(Euclid Collaboration: Salvalaggio et al., in prep.)\xspace}$
$\newcommand{\citelee}{Euclid Collaboration: Lee et al. (in prep.)\xspace}$
$\newcommand{\citeplee}{(Euclid Collaboration: Lee et al., in prep.)\xspace}$
$\newcommand{\citebruton}{Euclid Collaboration: Bruton et al. (in prep.)\xspace}$
$\newcommand{\citepbruton}{(Euclid Collaboration: Bruton et al., in prep.)\xspace}$
$\newcommand{\citepassa}{Euclid Collaboration: Passalacqua et al. (in prep.)\xspace}$
$\newcommand{\citeppassa}{(Euclid Collaboration: Passalacqua et al., in prep.)\xspace}$
$\newcommand{\citeapassa}{Euclid Collaboration: Passalacqua et al., in prep.\xspace}$
$\newcommand{\orcid}[1]$
$\newcommand{\linenumbers}[0]$
$\newcommand{\micron}{\mum\xspace}$
$\newcommand{\arraystretch}{1.12}$</div>



<div id="title">

# $\Euclid$ preparation: Controlling angular systematics in the ${\Euclid}$ spectroscopic galaxy sample

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.20856-b31b1b.svg)](https://arxiv.org/abs/2511.20856)<mark>Appeared on: 2025-11-27</mark> -  _submitted to A&A; 34 pages, 18 figures, 4 appendices_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>Y. Wang</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We present the strategy to identify and mitigate potential sources of angular  systematics in the _Euclid_ spectroscopic galaxy survey, and we quantify their  impact on galaxy clustering measurements and cosmological parameter estimation. We first  survey the _Euclid_ processing pipeline to identify all evident, potential  sources of systematics, and classify them into two broad classes: angular systematics,  which modulate the galaxy number density across the sky, and catastrophic redshift  errors, which lead to interlopers in the galaxy sample. We then use simulated  spectroscopic surveys to test our ability to mitigate angular systematics by  constructing a random catalogue that represents the visibility mask of the survey; this  is a dense set of intrinsically unclustered objects, subject to the same selection  effects as the data catalogue. The construction of this random catalogue relies on a  detection model, which gives the probability of reliably measuring the galaxy redshift  as a function of the signal-to-noise ratio (S/N) of its emission lines. We demonstrate  that, in the ideal case of a perfect knowledge of the visibility mask, the galaxy power  spectrum in the presence of systematics is recovered, to within sub-percent accuracy, by  convolving a theory power spectrum with a window function obtained from the random  catalogue itself. In the case of only approximate knowledge of the visibility mask, we  test the stability of power spectrum measurements and cosmological parameter posteriors  by using perturbed versions of the random catalogue. We find that significant effects  are limited to very large scales, and parameter estimation remains robust, with the most  impacting effects being connected to the calibration of the detection model.

</div>

<div id="div_fig1">

<img src="tmp_2511.20856/./Figures/shuf_pk_model.png" alt="Fig3" width="100%"/>

**Figure 3. -**  Upper panel: power spectra even multipoles ($\ell=0$, 2,
    4) of target galaxies in the $z\in[1.1,1.3]$ redshift bin, averaged over 50
    realisations, with shuffled fluxes to remove luminosity-dependent bias. Blue lines
    give the target measurement, while the lighter blue shaded areas give the error on an
    average of five measurements, covering an area similar to the final EWS. The red line
    gives the best fit model, convolved with the 30$^\circ$ cone window function. Lower
    panels give, for the three multipoles, the relative difference with respect to the
    model fit to the master mocks, convolved with the relative window. The pink shaded
    area highlights the region of the first BAO, while the dotted vertical lines mark the
    range of scales used for the fit. (*fig:pkmodel*)

</div>
<div id="div_fig2">

<img src="tmp_2511.20856/./Figures/detmodel.png" alt="Fig10.1" width="50%"/><img src="tmp_2511.20856/./Figures/detmodel_scatter.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** _Left:_ detection probability $P_{\rm det}(S)$, that is the probability
    of detecting a galaxy in the spectroscopic sample as a function of the S/N of its
    {\ha} line. The points give the measurements from the calibration set used to fix the
    parameters of the detection model. The five curves correspond to the best-fit
    detection model, together with those obtained by varying the two fitting parameters by
    1 $\sigma$(see the legend for the assumed parameter values). _Right:_ the
    standard deviation of the ratio of the measurements from the calibration set and the
    best fit model. (*fig:detmodel*)

</div>
<div id="div_fig3">

<img src="tmp_2511.20856/./Figures/shuf_pk3_realistic.png" alt="Fig16" width="100%"/>

**Figure 16. -** Monopole, quadrupole and hexadecapole of the galaxy power spectrum in the case
    of realistic mitigation, for MW extinction on the left, calibration error in the
    middle, and error on the detection model on the right. The lighter blue shaded areas
    give the variance of the target measurement, rescaled to the average of five
    catalogues. Here the $x$-axis scale is logarithmic. The residual panels report the
    ratio of the measurements with respect to the target one, all in per-cent. (*fig:pk_realistic*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.20856"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

135  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
